# Somraj Gautam (P20CS211) End semester-1

# 2nd part- Reproduced results on one of the databases used in the paper.(ImageNet-10)

## Contrastive Clustering (AAAI 2021)

### Link of the paper: - https://arxiv.org/pdf/2009.09687.pdf

### Github link used for implementation: - https://github.com/XLearning-SCU/2021-AAAI-CC

In [ ]:
!git clone https://github.com/XLearning-SCU/2021-AAAI-CC

Cloning into '2021-AAAI-CC'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 83 (delta 16), reused 5 (delta 5), pack-reused 58
Unpacking objects: 100% (83/83), done.
Checking out files: 100% (23/23), done.


In [2]:
%cd /content/drive/MyDrive/DL_assignment3/2021-AAAI-CC

/content/drive/MyDrive/DL_assignment3/2021-AAAI-CC


In [7]:
!pip install -q kaggle

In [9]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"somraj09","key":"997d1ba4e8a48a85fa75bb9e7f1ec34e"}'}

In [10]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
!kaggle datasets download -d liusha249/imagenet10

 98% 1.25G/1.27G [00:10<00:00, 134MB/s]
100% 1.27G/1.27G [00:10<00:00, 128MB/s]


### Imagenet-10 dataset downloading and extracting

In [12]:
!unzip /content/imagenet10.zip

Streaming output truncated to the last 5000 lines.
  inflating: imagenet-10/n04254680/n04254680_1389.JPEG  
  inflating: imagenet-10/n04254680/n04254680_13898.JPEG  
  inflating: imagenet-10/n04254680/n04254680_1395.JPEG  
  inflating: imagenet-10/n04254680/n04254680_13961.JPEG  
  inflating: imagenet-10/n04254680/n04254680_13987.JPEG  
  inflating: imagenet-10/n04254680/n04254680_1399.JPEG  
  inflating: imagenet-10/n04254680/n04254680_13994.JPEG  
  inflating: imagenet-10/n04254680/n04254680_1400.JPEG  
  inflating: imagenet-10/n04254680/n04254680_14007.JPEG  
  inflating: imagenet-10/n04254680/n04254680_1401.JPEG  
  inflating: imagenet-10/n04254680/n04254680_14028.JPEG  
  inflating: imagenet-10/n04254680/n04254680_14038.JPEG  
  inflating: imagenet-10/n04254680/n04254680_1406.JPEG  
  inflating: imagenet-10/n04254680/n04254680_14077.JPEG  
  inflating: imagenet-10/n04254680/n04254680_1411.JPEG  
  inflating: imagenet-10/n04254680/n04254680_14133.JPEG  
  inflating: imagenet-10/n04

In [14]:
%cd /content/drive/MyDrive/DL_assignment3/2021-AAAI-CC

/content/drive/MyDrive/DL_assignment3/2021-AAAI-CC


### For task 2 that is reproduce results on one of the databases used in the paper we used imagenet-10 dataset.

In [ ]:
!python train.py

Step [0/101]	 loss_instance: 5.5202836990356445	 loss_cluster: 2.9735615253448486
Step [50/101]	 loss_instance: 5.238405227661133	 loss_cluster: 2.683297872543335
Step [100/101]	 loss_instance: 5.266791343688965	 loss_cluster: 2.716977834701538
Epoch [1/100]	 Loss: 8.00054658757578
Step [0/101]	 loss_instance: 5.125730514526367	 loss_cluster: 2.6556949615478516
Step [50/101]	 loss_instance: 5.137907028198242	 loss_cluster: 2.669917583465576
Step [100/101]	 loss_instance: 5.173924922943115	 loss_cluster: 2.629389524459839
Epoch [2/100]	 Loss: 7.697283466263573
Step [0/101]	 loss_instance: 4.959355354309082	 loss_cluster: 2.5752570629119873
Step [50/101]	 loss_instance: 4.935802459716797	 loss_cluster: 2.5611765384674072
Step [100/101]	 loss_instance: 4.804286003112793	 loss_cluster: 2.525698661804199
Epoch [3/100]	 Loss: 7.47535238643684
Step [0/101]	 loss_instance: 4.890316486358643	 loss_cluster: 2.5877010822296143
Step [50/101]	 loss_instance: 4.786557197570801	 loss_cluster: 2.53850

In [3]:
!pip install munkres

### As the paper used Resnet34 as backbone model for fair comparison, we also setting resnet34 in while training

Accuracy after 20 epochs

In [24]:
import os
import argparse
import torch
import torchvision
import numpy as np
from torch.utils import data
import copy

def inference(loader, model, device):
    model.eval()
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            c = model.forward_cluster(x)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = torchvision.datasets.ImageFolder(
            root='/content/imagenet-10',
            transform=transform.Transforms(size=224).test_transform,
        )
    class_num = 10
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=500,
        shuffle=False,
        drop_last=False,
        num_workers=4,
    )

    res = resnet.get_resnet('ResNet34')
    model = network.Network(res, 128, class_num)
    model_fp = os.path.join('save/imagenet-10', "checkpoint_{}.tar".format('20')) #here epoch is set to 20
    model.load_state_dict(torch.load(model_fp, map_location=device.type)['net'])
    model.to(device)

    print("### Creating features from model ###")
    X, Y = inference(data_loader, model, device)
    nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print("NMI, ARI, F, ACC at 50 epoch") #but while printing it is still 50 written but accuracy is of 20 epoch
    print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))


### Creating features from model ###
Step [0/26]	 Computing features...
Step [20/26]	 Computing features...
Features shape (13000,)
NMI, ARI, F, ACC at 50 epoch
NMI = 0.4677 ARI = 0.3974 F = 0.4592 ACC = 0.6312


### Accuracy after 50 epochs

In [22]:
import os
import argparse
import torch
import torchvision
import numpy as np
from torch.utils import data
import copy


def inference(loader, model, device):
    model.eval()
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            c = model.forward_cluster(x)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = torchvision.datasets.ImageFolder(
            root='/content/imagenet-10',
            transform=transform.Transforms(size=224).test_transform,
        )
    class_num = 10
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=500,
        shuffle=False,
        drop_last=False,
        num_workers=4,
    )

    res = resnet.get_resnet('ResNet34')
    model = network.Network(res, 128, class_num)
    model_fp = os.path.join('save/imagenet-10', "checkpoint_{}.tar".format('50'))
    model.load_state_dict(torch.load(model_fp, map_location=device.type)['net'])
    model.to(device)

    print("### Creating features from model ###")
    X, Y = inference(data_loader, model, device)
    nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print("NMI, ARI, F, ACC at 50 epoch")
    print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))


### Creating features from model ###
Step [0/26]	 Computing features...
Step [20/26]	 Computing features...
Features shape (13000,)
NMI, ARI, F, ACC at 50 epoch
NMI = 0.6365 ARI = 0.5721 F = 0.6152 ACC = 0.7543


### Accuracy after 100 epoch

In [23]:
import os
import argparse
import torch
import torchvision
import numpy as np
from torch.utils import data
import copy


def inference(loader, model, device):
    model.eval()
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            c = model.forward_cluster(x)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = torchvision.datasets.ImageFolder(
            root='/content/imagenet-10',
            transform=transform.Transforms(size=224).test_transform,
        )
    class_num = 10
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=500,
        shuffle=False,
        drop_last=False,
        num_workers=4,
    )

    res = resnet.get_resnet('ResNet34')
    model = network.Network(res, 128, class_num)
    model_fp = os.path.join('save/CIFAR-10', "checkpoint_{}.tar".format('100'))
    model.load_state_dict(torch.load(model_fp, map_location=device.type)['net'])
    model.to(device)

    print("### Creating features from model ###")
    X, Y = inference(data_loader, model, device)
    nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print("NMI, ARI, F, ACC at 100 epoch")
    print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))


### Creating features from model ###
Step [0/26]	 Computing features...
Step [20/26]	 Computing features...
Features shape (13000,)
NMI, ARI, F, ACC at 100 epoch
NMI = 0.7329 ARI = 0.6991 F = 0.7292 ACC = 0.8310


***Above are the same NMI, ARI, F and Accuracy as that is in the original paper, thus task 2 is implemented completely.***